In [ ]:
######## MRes Thesis############
### Installing libraries
import pandas as pd
import html
import re
import requests
import os
import json
import csv
import datetime
import time
import dateutil.parser
import unicodedata
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tweepy
from textblob import TextBlob
print("Success installing libraries")

In [ ]:
##################### Writing functiion to access Twitter API ##################### 
# From https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

##### Passwords etc #####
# this line sets a TOKEN variable in the environment
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAGuQogEAAAAAZb5IMMyHKqkf7R9Roxf86bw3UsQ%3DBEr5sAMWj947VD288Lbj1XyUXVRFVU1WGWV9f4JQb4OjsggxYO' # Replace <ADD_BEARER_TOKEN> with your own token, then delete this line after running it the first time
consumer_key = "ZiuEdUJcu8oMS5R7f7oARJvEe" #change this to your consumer key
consumer_secret = "n4xV3vA9KT50w00lMFSpLBHYjqui2uuh2HlyNbXsIS6pb1pAob" #change this to your consumer secret

access_token =  "1126141054116405249-yBmJ4NL3lUwGJCHvDz9SQcWJaardkb" #change this to your access token
access_token_secret = "Cfuv3W4W4NbLbd3pBHR5Alj1q8O73EmmRWe1CLjxTyhGk" #change this to your access token secret

import os
import requests

#creating an auth() function that has the Bearer Token from Twitter

#create auth functiuon to retreive token from the environment
def auth():
    return os.getenv('TOKEN')

# Create headers. Define a function that takes the bearer token, pass it for auth, and return hearders we will use to access the API
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

#Then create URL. Now we can access teh API, we build the request for the endpointwe are going to use and the parameters we want to pass to the API
    #search_url is the link of the endpoint we want to access

############# Buidling the query
### Guide from https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
# Remember if you have basic or pro access then your query can only be 512 characters long for recent search endpoint. It's 1024 for premium and enterprise. Same with core and advance operators respectively.
# You have 3 targets to choose from: Low Traffic Neighbourhoods, 15 minute cities or frequent flier tax
LTN_search_query ="((Low traffic neighbourhood OR Low traffic neighbourhoods OR LTN OR LTNs OR liveable neighbourhood OR liveable neighbourhoods) lang:eng -airport)" #See https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query for how to build a query guide
FMC_target=("15-minute neighbourhood", "15-minute neighbourhoods", "20 minute neighbourhood", "20 minute neighbourhoods", "15 minute city", "15 minute cities", "20 minute city", "20 minute cities")
FFT_target=("frequent flyer tax", "frequent flyer levy", "progressive ticket tax", "frequent air miles levy", "air miles tax")


def create_url(keyword, start_date, end_date, max_results = 450):
    #max results is set by default at 10 but I have changed it to 100, the maximum. I am not including an end date. My keywod searhc is about LTNs, see line above

    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from. For this project it is recent search see https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
   
    #change params based on the endpoint you are using
    query_params = {'query': keyword, #one query for matching tweets.
                    'start_time': start_date,  
                    'end_time': end_date, #this is optional so I am commenting it out
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 
                    'tweet.fields': 'id,text,note_tweet,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {} #unique identifier to access the next page of requests 
                    }
    return (search_url, query_params)
    ############# connect to endpoint #############

#This creates a function that puts it all toether and connects to the endpoint
#This sends the GET requets if it si all conrrect and resutns the JSON response

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


 ##### defining my terms
#Inputs for the request
 # find your end time code using https://www.timestamp-converter.com/
bearer_token = auth()
headers = create_headers(bearer_token)
keyword= "(Low traffic neighbourhood OR Low traffic neighbourhoods OR LTN OR LTNs OR liveable neighbourhood OR liveable neighbourhoods) lang:en -airport -bank -is:retweet"
#keyword is your search query, see query builder

#tyring to make the start and end dates work!
from datetime import datetime, timedelta

# Specify the desired date (21st August 2023)
desired_start_date = datetime(2023, 8, 21)
# Specify the desired start hour (e.g. 10 is 10:00 AM). 0 represents midnight (12:00 AM), and 23 represents 11:00 PM. 
desired_start_hour = 0
desired_end_hour = 22

# Calculate the start and end times for that date
start_date = desired_start_date.replace(hour=desired_start_hour, minute=0, second=0)
end_date = desired_start_date.replace(hour=desired_end_hour, minute=0, second=0)
#end_date = desired_date + timedelta(days=1)  # End date is the next day

max_results = 10

print("finished creating an auth function new")

In [ ]:
############## Collect your tweets ##############

#If the returned response from the below code is 200, then the request was successful.
url = create_url(keyword, start_date_str, end_date_str, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

#print response in a readable format
print(json.dumps(json_response, indent=4, sort_keys=True))

In [ ]:
############# save the tweets in a file #############
#### as a JSON file 
from tweepy import Response
import requests
import os
from flask import Flask, render_template, Response

with open('response.json', 'w') as f:
    json.dump(json_response, f)

# must do this separately from your function so that it doesn't interfere with looping over requests
# Create file
csvFile = open("data1.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','tweet'])
csvFile.close()

def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        #source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

append_to_csv(json_response, "data1.csv")


In [ ]:
######## Time to loop through the requests! V3 ########
# This didn't work because I somehow managed to take out the for loop?? and it waswn't iterating over the time frame
# Set your desired maximum limit
desired_max_tweets = 150  # You can adjust this number as needed. 
#Set it as desired_max_tweet= the number you want to collect - max_results because of the way the loop works
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "(Low traffic neighbourhood OR Low traffic neighbourhoods OR LTN OR LTNs OR liveable neighbourhood OR liveable neighbourhoods) lang:en -airport -bank -is:retweet"
max_results = 100

from datetime import datetime, timedelta

# Calculate the start and end dates for a 7-day window
#end_date = datetime.utcnow() - timedelta(seconds=12)  # Adjust end_date to be at least 10 seconds earlier
#start_date = end_date - timedelta(days=6)  # 6 days before the end date

# Convert the dates to the required format
#end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:%SZ')
#start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:%SZ')

# Specify the desired date (21st August 2023)
desired_start_date = datetime(2023, 8, 24)
# Specify the desired start hour (e.g. 10 is 10:00 AM). 0 represents midnight (12:00 AM), and 23 represents 11:00 PM. 
desired_start_hour = 0
desired_end_hour = 13 # it goes from most recent tweets so you have to work backwards and change this number to avoid duplicates

# Calculate the start and end times for that date
start_date = desired_start_date.replace(hour=desired_start_hour, minute=0, second=0)
end_date = desired_start_date.replace(hour=desired_end_hour, minute=20, second=0)
#end_date = desired_date + timedelta(days=1)  # End date is the next day

# Convert the dates to the required format
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:%SZ')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:%SZ')


# Total number of tweets we collected from the loop
total_tweets = 0

# Create file and write headers
csvFile = open("dataFull2408AM.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['author id', 'created_at', 'geo', 'id', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'tweet'])
csvFile.close()

while total_tweets < desired_max_tweets:
    print("Loop Start")  # Print to indicate loop iteration start

    # Inputs for each loop iteration
    count = 0  # Counting tweets per time period
    max_count = 450  # Max tweets per time period
    next_token = None

    # Check if max_count reached
    while count < max_count and total_tweets < desired_max_tweets:
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_date_str, end_date_str, max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for the next call
            next_token = json_response['meta']['next_token']

        if result_count is not None and result_count > 0:
            append_to_csv(json_response, "dataFull2408AM.csv")
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("count is < than max_count(450) for this timeperiod: ", count)
            print("-------------------")

        time.sleep(5)
        print("Loop End for this API time period")  # Print to indicate loop iteration end
    print("-------------------")
    # Sleep for 15 minutes before the next iteration
    current_time = datetime.now() # Get current time
    next_iteration_time = current_time + timedelta(minutes=15) # Calculate time after 15 minutes
    #print("Current time:", current_time)
    #print("Sleeping for 15 minutes now. Next iteration will start at:", next_iteration_time)

    #time.sleep(15*60)  # Sleep for 15 minutes

print("Total number of results: ", total_tweets)


In [ ]:
#### Trying to save as much data as I can after my mistake ###
# I saved all my random data sets (from previously testing the code) in a single workbook
import pandas as pd
import re
from emoji import demojize
# Read the Excel file with multiple sheets
file_path = 'alldatasofarnotduplciatedpreviously.xlsx'
excel_file = pd.ExcelFile(file_path)

# Create a list to store cleaned data from all sheets
cleaned_data_list = []

# Function to clean the text
def clean_text(text):
    # Remove hyperlinks
    text = re.sub(r'http\S+', '', text)

    # Both \r\n and \n can appear in your tweets. 
    # When creating a CSV from the JSON response, however, this can cause problems. 
    #Renove \r and repalce it with a space
    text = re.sub(r'\r',' ', text)
    
    # Remove new lines
    text = text.replace('\n', ' ')
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    
    # Remove usernames (starting with @)
    text = re.sub(r'@\w+', '', text)
    
    # Remove emojis
    text = demojize(text)

    #Remove backslashes followed by quotes
    text = re.sub(r'\\\"', r'"', text)
    
    # Remove other special characters
    text = re.sub(r'\\', '', text)

    # Replace HTML character entities
    text = re.sub(r'&amp;', '&', text)
    
    #make everything lowercase
    text= text.lower()
    return text.strip()


# Loop through each sheet in the Excel file
for sheet_name in excel_file.sheet_names:
    # Read the sheet into a DataFrame
    sheet_data = excel_file.parse(sheet_name)
    
    # Drop columns that don't have specified titles
    required_columns = ['author id', 'created_at', 'geo', 'id', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'tweet']
    cleaned_sheet_data = sheet_data[required_columns]
    
    # Clean the "tweet" column and create the "text" column
    #cleaned_sheet_data['clean_tweet'] = cleaned_sheet_data['tweet'].apply(clean_text)

    # Drop duplicates based on the 'tweet' column 
    cleaned_sheet_data = cleaned_sheet_data.drop_duplicates(subset='tweet', keep='first')
    
    # Append cleaned data to the list
    cleaned_data_list.append(cleaned_sheet_data)

# Concatenate all cleaned data into a single DataFrame
combined_cleaned_data = pd.concat(cleaned_data_list, ignore_index=True)

# Save combined cleaned data to a CSV file
output_file_path = 'finalcombineddatasetnoduplicates.csv'
combined_cleaned_data.to_csv(output_file_path, index=False)

print("Combined cleaned data saved to:", output_file_path)


In [ ]:
################### Cleaning up my data ################### V2
# Some tips have been taken from here https://towardsdatascience.com/mistakes-to-avoid-when-using-twitter-data-for-the-first-time-304c3d0ef7a6
# Some text will be contained in images or video, which I will not be able to access
import pandas as pd
import re
import demoji
from emoji import demojize
from nltk.stem import PorterStemmer  # Import the stemmer

demoji.download_codes() # you need to download emojis
#When I check the data in excel and R, there are still some duplciates. I'm not sure why, and I can't seem to make them disappear here.
#So I removed them manually in Excel and checked on python.
#The new file is called #unique_combined_cleaned_data.csv

# Load the CSV file
csv_file_path = "finalcombineddatasetnoduplicates_duplciatesbetweendatasetsremoved.csv"
data = pd.read_csv(csv_file_path, encoding='utf-8')

# Function to clean the text
def clean_text(text):
    # Remove hyperlinks
    text = re.sub(r'http\S+', '', text)

    # Both \r\n and \n can appear in your tweets. 
    # When creating a CSV from the JSON response, however, this can cause problems. 
    #Renove \r and repalce it with a space
    text = re.sub(r'\r',' ', text)
    
    # Remove new lines
    text = text.replace('\n', ' ')
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    
    # Remove usernames (starting with @)
    text = re.sub(r'@\w+', '', text)
    
    #Remove backslashes followed by quotes
    text = re.sub(r'\\\"', r'"', text)
    
    # Remove other special characters
    text = re.sub(r'\\', '', text)

    # Replace HTML character entities
    text = re.sub(r'&amp;', '&', text)
    return text.strip()

# Clean the "tweet" column and create the "text" column
data['text'] = data['tweet'].apply(clean_text)

# Stemming function using Porter stemmer
stemmer = PorterStemmer()  # Initialize the stemmer

def stem_text(text):
    text = demojize(text) #convert emojis to text
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Duplicate the 'text' column and apply stemming
data['stemmed_text'] = data['text'].apply(stem_text)

# Continue with the rest of your code...

# Save the updated data back to the CSV file
cleaned_csv_file_path = "cleaned_data.csv"
data.to_csv(cleaned_csv_file_path, index=False,  encoding='utf-8')

print("Cleaning complete. Data saved to cleaned_data.csv.")

In [ ]:
####  Trying to remove weird encodings
# I'm getting werid encodings because UTF-8 encodes special characters using 1 to 4 bytes. These have been split up and decoded into
# Western, before being encoded again. They're also of variable length so I need to find a way to remove them.
# https://stackoverflow.com/questions/31671906/how-do-i-recreate-these-special-characters-when-testing-a-website
import re
import pandas as pd

#These I have picked up manually by looking at tweets
text = re.sub("ÃƒÂ¢Ã¢â€šÂ¬Ã¢â€žÂ¢", "'", text)
text = re.sub("RT :", "", text)
text = re.sub("ÃƒÂ¢Ã‹Å“Ã‚ÂºÃƒÂ¯Ã‚Â¸Ã‚Â", ":)", text)
text = re.sub("ÃƒÂ°Ã…Â¸Ã¢â‚¬ËœÃ¢â‚¬Â°", "Search instead for", text)
text = re.sub("ÃƒÂ¢Ã…â€œÃ¢â‚¬Â¦", "tick emoji", text)
text = re.sub("Ãƒâ€šÃ‚Â", "", text)
text = re.sub("ÃƒÂ°Ã…Â¸Ã‹Å“Ã‚Â£", ":(", text)

#### Making a function to remove all other weird encodings
import unicodedata
#def remove_weird_sequences(text):
 #   cleaned_text = re.sub(r'[\u0080-\uFFFF]', '', text)
  #  cleaned_text = re.sub(r'ÃƒÂ+', '', cleaned_text)
    #cleaned_text = re.sub(r'ƒ+', '', cleaned_text)
   # return cleaned_text

def remove_weird_sequences(text):
    if pd.notna(text):
        cleaned_text = re.sub(r'[\u0080-\uFFFF]', '', text)
        cleaned_text = re.sub(r'ÃƒÂ+', '', cleaned_text)
        cleaned_text = re.sub(r'ƒ+', '', cleaned_text)
        return cleaned_text
    else:
        return ""

# Example usage
text1 = "This is a text with weird sequences ÃƒÂabcÃƒÂdef ÃƒÂghi and some punctuation: ÃƒÂ; , !"
cleaned_text1 = remove_weird_sequences(text1)
print(cleaned_text1)  # Output: "This is a text with weird sequences and some punctuation:; , !"

text2 = "The MayorÃƒÂ¢Ã¢â€šÂ¬Ã¢â€žÂ¢s car!"
cleaned_text2 = remove_weird_sequences(text2)
print(cleaned_text2)  # Output: "The Mayors car!"


#Time to apply it 

csv_file_path = "cleaned_data.csv"
data = pd.read_csv(csv_file_path, encoding='utf-8')

data['text'] = data['text'].apply(remove_weird_sequences)
data['stemmed_text'] = data['stemmed_text'].apply(remove_weird_sequences)

# Save the updated data back to the CSV file
cleaned_csv_file_path = "cleaned_data_utf8.csv"
data.to_csv(cleaned_csv_file_path, index=False,  encoding='utf-8')

print("Cleaning complete. Data saved to cleaned_data.csv.")

In [ ]:
#### Checking for duplicates
# Check for duplicate tweet content
duplicate_content = data[data.duplicated('text', keep=False)]

# Check for duplicate tweet IDs
duplicate_ids = data[data.duplicated('author id', keep=False)]

print("Duplicate tweets based on content:")
print(duplicate_content)

#print("\nDuplicate tweets based on tweet IDs:")
#print(duplicate_ids)

In [ ]:
################### Detecting moral foundations in tweets ###################

#### This uses the MoralStrength Lexicon
#### Taken from https://github.com/oaraque/moral-foundations ####

# WARNING: You will get warnings about inconsistent versions of scikit learn and spaCy.
# This is often realted to the compatibiliyt of the library version you haev installed.
# These warnging alert you that you are using models trained wi th different versions of the library which can lead to invalid results. 
# I have tried to resolve these woarning by ipgrading my library versions and updating dependencies
# I do n't have the time to repickle my models using the latest versions
# You can turn off these warnings using the code below
#import warnings
#from sklearn.exceptions import InconsistentVersionWarning
#warnings.filterwarnings("ignore", category=InconsistentVersionWarning)

#tesing out the lexicon
import moralstrength
from moralstrength.moralstrength import estimate_morals

#### Here is example code from the github page so you can see how it works ####
text = "PLS help #HASHTAG's family. No one prepares for this. They are in need of any assistance you can offer"  

result= moralstrength.string_moral_value(text, moral='care')
print(result)

from moralstrength.moralstrength import estimate_morals

texts = '''My dog is very loyal to me.
My cat is not loyal, but understands my authority.
He did not want to break the router, he was fixing it.
It is not fair! She cheated on the exams.
Are you pure of heart? Because I am sure not.
Will you take care of me? I am sad.'''

texts = texts.split('\n')

result = estimate_morals(texts, process=True) # set to false if text is alredy pre-processed
print(result)

import moralstrength

text = "PLS help #HASHTAG's family. No one prepares for this. They are in need of any assistance you can offer"  

moralstrength.string_moral_value(text, moral='care')

In [ ]:
#### Updating lexicon to most recent and to include liberty ###
import pandas as pd
# Update versiom
from moralstrength import lexicon_use
lexicon_use.select_version("latest")

#Attempting to add the liberty lexicon https://github.com/oaraque/moral-foundations/blob/master/liberty/2nd_version/cs_lexicon_final.csv
# Load the liberty lexicon CSV file of your choice
lexicon_file_path = "we_lexicon_final.csv" #"cs_lexicon_final.csv"  # Or "we_lexicon_final.csv"
lexicon_data = pd.read_csv(lexicon_file_path)

# Display the first few rows of the lexicon DataFrame
print(lexicon_data.head())

#### Trying to estimate liberty with a function of our own ####
# Load the word embedding lexicon
lexicon_path = 'we_lexicon_final.csv'
lexicon = pd.read_csv(lexicon_path)

def estimate_liberty(texts):
    """
    Estimate Liberty moral values for a list of texts using the provided word embedding lexicon.

    Parameters:
    - texts: List of texts to estimate Liberty moral values for.

    Returns:
    - A Pandas DataFrame with the estimated Liberty moral values for each input text.
    """

    liberty_scores = []  # List to store Liberty scores

    for text in texts:
        liberty_score = 0  # Initialize the Liberty score
        
        for word in text.split():  # Split text into words (adjust as needed)
            word_data = lexicon[lexicon['word'] == word]
            if not word_data.empty:
                liberty_score += word_data['score'].values[0]
        
        liberty_scores.append(liberty_score)

    # Create a DataFrame with the Liberty scores
    estimation = pd.DataFrame({'liberty': liberty_scores})

    return estimation

# Example usage
texts = ["Freedom is essential for a thriving society.",
         "Government should have limited power over individuals.",
         "Liberty ensures individual rights and autonomy."]
liberty_estimation = estimate_liberty(texts)
print(liberty_estimation)


# Example usage
texts = ["I love LTN",
         "Low Traffic Neighbourhood 'discourse' has made it into the local Facebook group. It's full of all the usual mad nonsense you'd come to expect, depressingly",
         "We don't live in a democracy. Only the illusion if one. Net zero no one asked. LTN'S no one asked ULEZ no one asked. Speed bumps no one asked. Mass immigration no one asked."]
liberty_estimation = estimate_liberty(texts)
print(liberty_estimation)

#something isn't right here because the first sentence here should not be scoring.

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from lexicon_use import form_text_vector
from estimators import estimate

from gsitk.preprocess import pprocess_twitter, simple, Preprocessor


#Dont have time to make this workd so I will give up on including Liberty

In [ ]:
from moralstrength import lexicon_use
from moralstrength.estimators import estimate,models
from moralstrength.moral_list import moral_options_lexicon, moral_options_predictions
from moralstrength import string_moral_values
#text='LTNs are the wrost. Anyone who likes them is my enemy'
#text='LTNs are the best. I love clean air'
#text='It brought our neighbourhood closer together - we all had posters in our windows saying "Bollards to LTNs" and we all went on marches to the town hall together.'
text='PLS help #HASHTAGs family. No one prepares for this. They are in need of any assistance you can offer'
string_moral_values(text, model='unigram+freq')

In [ ]:
# Import necessary libraries
import pandas as pd
from moralstrength import lexicon_use
from moralstrength.estimators import estimate,models
from moralstrength.moral_list import moral_options_lexicon, moral_options_predictions
from moralstrength import string_moral_values


#moralstrength.string_moral_value(text, moral='care')
# Load your cleaned CSV file into a DataFrame
cleaned_csv_file_path = "cleaned_data_utf8.csv"
data = pd.read_csv(cleaned_csv_file_path)

# Define a list of moral traits you want to estimate
moral_traits = ["loyalty", "authority", "purity", "fairness", "care"]

# Function to estimate moral strengths for a given tweet
def estimate_moral_strengths(tweet):
    moral_values = string_moral_values(tweet, model="unigram+freq")
    return moral_values

# Iterate through the moral traits and create new columns for moral strengths
for trait in moral_traits:
    data[trait + "_strength"] = data['tweet'].apply(lambda x: estimate_moral_strengths(x).get(trait, None))
# Save the updated data with individualizing and moral_present values to a new CSV file
data_with_values_csv_file_path = "data_with_moral_values.csv"
data.to_csv(data_with_values_csv_file_path, index=False)

print("Individualizing and moral_present values added to the data. Data saved to", data_with_values_csv_file_path)

In [ ]:
### V2 Detecting presenece of moral values and their strenghth in tweets ##########
from moralstrength import estimate_morals, lexicon_morals
# Load your cleaned CSV file into a DataFrame
cleaned_csv_file_path = "cleaned_data_utf8.csv"
data = pd.read_csv(cleaned_csv_file_path)

# Estimate moral values for each tweet and create new columns for each moral trait
def estimate_moral_values(tweet):
    moral_values = estimate_morals([tweet], process=True)
    return moral_values.iloc[0]

moral_columns = lexicon_morals()
for column in moral_columns:
    data[column] = data['stemmed_text'].apply(lambda x: estimate_moral_values(x)[column] if pd.notnull(x) else None)

# Calculate individualizing values for each row and create a new column
def classify_individualizing(row):
    if (row['fairness'] > 0) or (row['care'] > 0):
        return 1  # Fairness or care is present
    else:
        return 0  # Other cases
    
def classify_binding(row):
    if (row['loyalty'] > 0) or (row['authority'] > 0) or (row['purity'] > 0):
        return 1  # Loyalty, authority, or purity is present
    else:
        return 0  # Other cases

def classify_individualizing_cont(row):
    fairness = row['fairness'] if not pd.isna(row['fairness']) else 0
    care = row['care'] if not pd.isna(row['care']) else 0
    if (fairness > 0) or (care > 0):
        return (fairness + care)
    else:
        return 0

def classify_binding_cont(row):
    loyalty = row['loyalty'] if not pd.isna(row['loyalty']) else 0
    authority = row['authority'] if not pd.isna(row['authority']) else 0
    purity = row['purity'] if not pd.isna(row['purity']) else 0
    if (loyalty > 0) or (authority > 0) or (purity > 0):
        return (loyalty + authority + purity)
    else:
        return 0
    
def classify_bin_individualizing(row):
    if (row['fairness'] > 0) or (row['care'] > 0):
        return 2  # Fairness or care is present
    elif (row['loyalty'] > 0) or (row['authority'] > 0) or (row['purity'] > 0):
        return 1 # Binding is present
    else:
        return 0  # no morals are pesent

#apply these functions to the data
data['individualising_cat'] = data.apply(classify_individualizing, axis=1)
data['binding_cat'] = data.apply(classify_binding, axis=1)
data['individualising_cont'] = data.apply(classify_individualizing_cont, axis=1)
data['binding_cont'] = data.apply(classify_binding_cont, axis=1)
data['binary_indiv'] = data.apply(classify_bin_individualizing, axis=1)

# Calculate moral_present values for each row and create a new column
def classify_moral_present(row):
    for column in moral_columns:
        if column != 'liberty' and row[column] > 0:
            return 1  # At least one moral (excluding liberty) is present
    return 0  # No moral (excluding liberty) is present

data['moral_present'] = data.apply(classify_moral_present, axis=1)

# Save the updated data with individualizing and moral_present values to a new CSV file
data_with_values_csv_file_path = "data_with_moral_values.csv"
data.to_csv(data_with_values_csv_file_path, index=False)

print("Individualizing and moral_present values added to the data. Data saved to", data_with_values_csv_file_path)

In [6]:
########### Sentiment analysis - VADER ######

# compound score is computed by summing the valence scores of each word 
#in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). 
#This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence.
# Calling it a 'normalized, weighted composite score' is accurate.
#It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either 
#positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:
# positive sentiment: compound score >= 0.05
# neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
# negative sentiment: compound score <= -0.05
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load your cleaned CSV file into a DataFrame
cleaned_csv_file_path = "data_with_moral_values.csv"
data = pd.read_csv(cleaned_csv_file_path)

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment based on compound score
def classify_sentiment(compound_score):
    if compound_score >= 0.05:
        return 2  # Positive sentiment
    elif -0.05 < compound_score < 0.05:
        return 1  # Neutral sentiment
    else:
        return 0  # Negative sentiment

# Filter out NaN values from the 'text' column
data = data.dropna(subset=['text'])

# Calculate sentiment for each text and create a new column
data['VADER_cont'] = data['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
data['VADER_cat'] = data['VADER_cont'].apply(classify_sentiment)

# Save the updated data with sentiment values to a new CSV file
data_with_sentiment_csv_file_path = "data_with_sentiment.csv"
data.to_csv(data_with_sentiment_csv_file_path, index=False)

print("Sentiment values added to the data. Data saved to", data_with_sentiment_csv_file_path)


Sentiment values added to the data. Data saved to data_with_sentiment.csv


In [ ]:
######### Experimenting with word clouds####
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


# Filter data based on sentiment (replace 'positive' and 'negative' with your actual column names)
positive_text = " ".join(data[data['VADER_cat'] == 2]['text'])
negative_text = " ".join(data[data['VADER_cat'] == 0]['text'])

# Initialize WordCloud object with custom stopwords
stopwords = set(STOPWORDS)
#stopwords.update(['br', 'film', 'movie'])  # Update stopwords
wc = WordCloud(background_color='white', max_words=50, stopwords=stopwords)

# Generate and plot word cloud for positive sentiment
positive_wordcloud = wc.generate(positive_text)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(positive_wordcloud, interpolation='bilinear')
plt.title('Positive Sentiment')
plt.axis('off')

# Generate and plot word cloud for negative sentiment
negative_wordcloud = wc.generate(negative_text)
plt.subplot(1, 2, 2)
plt.imshow(negative_wordcloud, interpolation='bilinear')
plt.title('Negative Sentiment')
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
######### Experimenting with word clouds####
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


# Filter data based on sentiment (replace 'positive' and 'negative' with your actual column names)
liberal_text = " ".join(data[data['binary_indiv'] == 2]['text'])
conservative_text = " ".join(data[data['binary_indiv'] == 1]['text'])

# Initialize WordCloud object with custom stopwords
stopwords = set(STOPWORDS)
#stopwords.update(['br', 'film', 'movie'])  # Update stopwords
wc = WordCloud(background_color='white', max_words=50, stopwords=stopwords)

# Generate and plot word cloud for positive sentiment
Liberal_wordcloud = wc.generate(liberal_text)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(Liberal_wordcloud, interpolation='bilinear')
plt.title('Liberal')
plt.axis('off')

# Generate and plot word cloud for negative sentiment
Conservative_wordcloud = wc.generate(conservative_text)
plt.subplot(1, 2, 2)
plt.imshow(Conservative_wordcloud, interpolation='bilinear')
plt.title('Conservative')
plt.axis('off')

plt.tight_layout()
plt.show()